# 3. Feature Engineering

Bu notebook yeni feature turetme ve feature selection surecini icerir.

In [ ]:
# Kutuphaneler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score

# LightGBM
import lightgbm as lgb

# Sabitler
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('Kutuphaneler yuklendi!')

## Veri Yukleme ve baseline skor

In [ ]:

DATA_PATH = Path('../data/raw/')
df = pd.read_csv(DATA_PATH / 'bank.csv')
# Baseline skor (onceki notebook'tan)

BASELINE_AUC = 0.9259  # Test seti AUC
BASELINE_CV_AUC = 0.9239  # Cross-validation AUC

print(f'Veri yuklendi: {df.shape[0]:,} satir, {df.shape[1]} sutun')
print(f'\nBaseline Skorlari:')
print(f'  CV AUC: {BASELINE_CV_AUC:.4f}')
print(f'  Test AUC: {BASELINE_AUC:.4f}')

In [ ]:

TARGET = 'deposit'
y = (df[TARGET] == 'yes').astype(int)
X = df.drop(columns=[TARGET])

# Kategorik ve numerik sutunlar
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f'Kategorik sutunlar: {len(cat_cols)}')
print(f'Numerik sutunlar: {len(num_cols)}')

# label encoding (baseline preprocessing)
X_baseline = X.copy()
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X_baseline[col] = le.fit_transform(X_baseline[col].astype(str))
    label_encoders[col] = le

print(f'\nBaseline feature sayisi: {X_baseline.shape[1]}')

In [ ]:
# Feature engineering başlangıcı
X_fe = X.copy()

# 1. DURATION'ı ÇIKAR (production'da bilemeyiz)
print("1. duration feature'i çıkarılır (production'da bilemiyoruz)...")
X_fe = X_fe.drop(columns=['duration'])
print(f"   Feature sayısı: {X_fe.shape[1]}")

# 2. YAS GRUPLARI
print("\n 2. Yas grupları oluşturuluyor")
X_fe['age_group'] = pd.cut(
    X_fe['age'], 
    bins=[0, 30, 40, 50, 60, 100],
    labels=['18-30', '31-40', '41-50', '51-60', '60+']
)
print(f"   Yas grupları: {X_fe['age_group'].value_counts().to_dict()}")

# 3. BAKIYE KATEGORİLERİ
print("\n3. Bakiye kategorileri oluşturuluyor...")
X_fe['balance_category'] = pd.cut(
    X_fe['balance'],
    bins=[-np.inf, 0, 100, 500, 2000, np.inf],
    labels=['Negatif', 'Dusuk', 'Orta', 'Yuksek', 'Cok Yuksek']
)
print(f"   Bakiye kategorileri: {X_fe['balance_category'].value_counts().to_dict()}")

# 4. PDAYS FLAG (hiç aranmadı mı?)
print("\n4. pdays flag oluşturuluyor...")
X_fe['never_contacted'] = (X_fe['pdays'] == -1).astype(int)
print(f"   Hiç aranmamış müşteri: {X_fe['never_contacted'].sum():,} ({(X_fe['never_contacted'].mean()*100):.1f}%)")

print(f"\nFaz 1 tamamlandı! Yeni feature sayısı: {X_fe.shape[1]}")

### Faz I'de Yapılanlar

1. **duration Feature'ini Çıkarma**
   - `duration` (görüşme süresi) feature'i production'da bilinmez
   - Görüşme yapılmadan önce bu bilgiye sahip olunamaz
   - Gerçekçi bir model için bu feature çıkarıldı
   - **Etki:** Feature sayısı 16'dan 15'e düştü

2. **Yas Grupları Oluşturma**
   - Yaş değişkenini kategorilere ayırdık: 18-30, 31-40, 41-50, 51-60, 60+
   - Modelin yaş grupları arasındaki farkları daha iyi öğrenmesini sağlar
   - **Yeni Feature:** `age_group` (kategorik)

3. **Bakiye Kategorileri Oluşturma**
   - Bakiye değişkenini anlamlı kategorilere ayırdık:
     - Negatif (borçlu)
     - Düşük (0-100)
     - Orta (100-500)
     - Yüksek (500-2000)
     - Çok Yüksek (2000+)
   - **Yeni Feature:** `balance_category` (kategorik)

4. **"Hiç Aranmadı" Flag'i**
   - `pdays = -1` değeri "daha önce hiç aranmamış" anlamına gelir
   - Bu özel durumu ayrı bir binary feature olarak işaretledik
   - **Yeni Feature:** `never_contacted` (0 veya 1)

**Toplam:** 3 yeni feature eklendi, 1 feature çıkarıldı → Net: 2 feaature

FAZ-I preprocessing ve performans ölçümü


In [ ]:
# Yardımcı fonksiyon: Feature setinin performansını ölçer
def evaluate_features(X, y, model_name='LightGBM', baseline_auc=BASELINE_CV_AUC):
    """Feature setinin performansını cross-validation ile ölçer."""
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    
    model = lgb.LGBMClassifier(
        n_estimators=100,
        learning_rate=0.1,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1
    )
    
    scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
    
    print(f'{model_name} CV AUC: {scores.mean():.4f} (+/- {scores.std():.4f})')
    
    if baseline_auc > 0:
        diff = scores.mean() - baseline_auc
        diff_pct = (diff / baseline_auc) * 100
        if diff > 0:
            print(f'Baseline ile fark: +{diff:.4f} (+{diff_pct:.2f}%) ✅')
        else:
            print(f'Baseline ile fark: {diff:.4f} ({diff_pct:.2f}%)')
    
    return scores.mean()

print('evaluate_features fonksiyonu tanımlandı!')

In [ ]:
# Faz 1: Kategorik feature'ları encode et
X_fe1 = X_fe.copy()

# Yeni kategorik feature'ları da ekle
cat_cols_fe1 = X_fe1.select_dtypes(include=['object', 'category']).columns.tolist()

# Label Encoding
label_encoders_fe1 = {}
for col in cat_cols_fe1:
    le = LabelEncoder()
    X_fe1[col] = le.fit_transform(X_fe1[col].astype(str))
    label_encoders_fe1[col] = le

print(f'Faz 1 - Toplam feature sayısı: {X_fe1.shape[1]}')
print(f'Kategorik feature\'lar encode edildi.\n')

# Performans ölçümü
print('=' * 50)
print('FAZ 1 PERFORMANS ÖLÇÜMÜ')
print('=' * 50)
faz1_auc = evaluate_features(X_fe1, y, 'Faz 1 (duration çıkarıldı + yeni feature\'lar)', BASELINE_CV_AUC)

FAZ-II ileri seviye feature lar

In [ ]:
# Faz 2: İleri seviye feature'lar
# ÖNEMLİ: X_fe kullan (henüz encode edilmemiş, month string olarak duruyor)
X_fe2 = X_fe.copy()

print("=" * 50)
print("FAZ 2: İLERİ SEVİYE FEATURE'LAR")
print("=" * 50)

# 1. MEVSİMSELLİK FEATURE'LARI (encode etmeden önce!)
print("\n1. Mevsimsellik feature'ları oluşturuluyor...")
month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
X_fe2['month_numeric'] = X_fe2['month'].map(month_map)

# Mevsim (quarter)
X_fe2['quarter'] = ((X_fe2['month_numeric'] - 1) // 3) + 1


In [ ]:
# Yıl sonu/başı (Q4 ve Q1 önemli olabilir)
X_fe2['is_year_end'] = (X_fe2['month_numeric'].isin([11, 12])).astype(int)
X_fe2['is_year_start'] = (X_fe2['month_numeric'].isin([1, 2])).astype(int)

print(f"   - month_numeric: Ay sayısal değer")
print(f"   - quarter: Çeyrek (1-4)")
print(f"   - is_year_end: Yıl sonu (Kasım-Aralık)")
print(f"   - is_year_start: Yıl başı (Ocak-Şubat)")

# 2. KAMPANYA YOĞUNLUĞU METRİKLERİ
print("\n2. Kampanya yoğunluğu metrikleri oluşturuluyor")

# Toplam iletişim sayısı (bu kampanya + önceki kampanyalar)
X_fe2['total_contacts'] = X_fe2['campaign'] + X_fe2['previous']

# kampanya başına ortalama iletişim (eğer daha önce aranmışsa)
X_fe2['avg_contacts_per_campaign'] = X_fe2['previous'] / (X_fe2['previous'] + 1)

# cok fazla aranmış mı? (outlier tespiti)
X_fe2['over_contacted'] = (X_fe2['campaign'] > 5).astype(int)

print(f"   - total_contacts: Toplam iletişim sayısı")
print(f"   - avg_contacts_per_campaign: Kampanya başına ortalama")
print(f"   - over_contacted: 5'ten fazla aranmış mı?")

# 3. İNTERAKSİYON FEATURE'LARI
print("\n 3. İnteraksiyon feature'ları oluşturuluyor...")

# Yaş x Bakiye (genç ve zengin vs yaşlı ve fakir)
X_fe2['age_balance_interaction'] = X_fe2['age'] * (X_fe2['balance'] / 1000)

# Yaş x Kampanya (yaşlılar daha fazla aranmış mı?)
X_fe2['age_campaign_interaction'] = X_fe2['age'] * X_fe2['campaign']

# Bakiye x Kampanya (zenginler daha fazla aranmış mı?)
X_fe2['balance_campaign_interaction'] = X_fe2['balance'] * X_fe2['campaign']

print(f"   - age_balance_interaction: Yaş x Bakiye")
print(f"   - age_campaign_interaction: Yaş x Kampanya")
print(f"   - balance_campaign_interaction: Bakiye x Kampanya")

# 4. RATIO FEATURE'LARI
# Yaş başına bakiye (ne kadar birikim var?)
print("\n4. Ratio feature'ları oluşturuluyor...")


X_fe2['balance_per_age'] = X_fe2['balance'] / (X_fe2['age'] + 1)

# Kampanya başına başarı oranı (eğer daha önce aranmışsa)
# poutcome'u numerik yapalım (success=1, diğerleri=0)
X_fe2['previous_success'] = (X_fe2['poutcome'] == 2).astype(int)  # LabelEncoder'da success=2 olabilir

print(f"   - balance_per_age: Yaş başına bakiye")
print(f"   - previous_success: Önceki kampanyada başarılı mı?")

print(f"\n Faz 2 tamamlandı! Yeni feature sayısı: {X_fe2.shape[1]}")
print(f"   Toplam eklenen feature: {X_fe2.shape[1] - X_fe.shape[1]}")

### Faz-II yapılanlar

1. **Mevsimsellik Feature'ları**
   - Ay'ı sayısal değere çevirdik (month_numeric: 1-12)
   - Çeyrek (quarter) oluşturduk 1-4
   - Yıl sonu flag'i (Kasım-Aralık)
   - Yıl başı flag'i (Ocak-Şubat)
   - **Amaç:** Mevsimsel trendleri yakalamak

2. **Kampanya Yoğunluğu Metrikleri**
   - Toplam iletişim sayısı (bu kampanya + önceki kampanyalar)
   - Kampanya başına ortalama iletişim
   - "Çok fazla aranmış" flag'i (5'ten fazla arama)
   - **Amaç:** Müşteri iletişim geçmişini özetlemek

3. **İnteraksiyon Feature'ları**
   - Yaş × Bakiye (genç-zengin vs yaşlı-fakir)
   - Yaş × Kampanya (yaşlılar daha fazla aranmış mı?)
   - Bakiye × Kampanya (zenginler daha fazla aranmış mı?)
   - **Amaç:** Feature'lar arası ilişkileri yakalamak

4. **Ratio Feature'ları**
   - Yaş başına bakiye (birikim oranı)
   - Önceki kampanyada başarılı mı? (binary flag)
   - **Amaç:** Normalize edilmiş metrikler oluşturmak

**Toplam:** ~10 yeni feature eklendi

FAZ-II Performans ölçümü


Faz 2: Eksik kontrolleri


In [ ]:
print("Eksik değer kontrolü...")
missing = X_fe2.isnull().sum()
if missing.sum() > 0:
    print(f"Eksik değerler bulundu:")
    print(missing[missing > 0])
    # Eksik değerleri doldur (numerik için median)
    for col in X_fe2.select_dtypes(include=[np.number]).columns:
        if X_fe2[col].isnull().sum() > 0:
            X_fe2[col] = X_fe2[col].fillna(X_fe2[col].median())
    print("Eksik değerler dolduruldu.")
else:
    print("Eksik değer yok")

In [ ]:
# Faz 2: Kategorik feature'ları encode et (tüm feature engineering'den sonra)
print("\nKategorik feature'lar encode ediliyor...")
cat_cols_fe2 = X_fe2.select_dtypes(include=['object', 'category']).columns.tolist()

label_encoders_fe2 = {}
for col in cat_cols_fe2:
    le = LabelEncoder()
    X_fe2[col] = le.fit_transform(X_fe2[col].astype(str))
    label_encoders_fe2[col] = le

print(f"   {len(cat_cols_fe2)} kategorik feature encode edildi.")
print(f"   Final feature sayısı: {X_fe2.shape[1]}")


In [ ]:
# Sonsuz değerleri kontrol et
print("\nSonsuz değer kontrolü...")
inf_cols = []
for col in X_fe2.select_dtypes(include=[np.number]).columns:
    if np.isinf(X_fe2[col]).any():
        inf_cols.append(col)
        X_fe2[col] = X_fe2[col].replace([np.inf, -np.inf], np.nan)
        X_fe2[col] = X_fe2[col].fillna(X_fe2[col].median())

if inf_cols:
    print(f"Sonsuz değerler düzeltildi: {inf_cols}")
else:
    print("Sonsuz değer yok")



In [ ]:
print(f'\nFaz 2 - Final feature sayısı: {X_fe2.shape[1]}')
print(f'Faz 1\'den eklenen: {X_fe2.shape[1] - X_fe1.shape[1]} feature\n')

# Performans ölçümü
print('=' * 50)
print('FAZ 2 PERFORMANS ÖLÇÜMÜ')
print('=' * 50)
faz2_auc = evaluate_features(X_fe2, y, 'Faz 2 (Tüm yeni feature\'lar)', faz1_auc)

Feature Selection

In [ ]:
# Feature Selection
print("=" * 50)
print("FEATURE SELECTION")
print("=" * 50)

X_fe_selected = X_fe2.copy()

# 1. Düşük varyanslı feature'ları çıkar
print("\n1. Düşük varyanslı feature'lar çıkarılıyor...")
selector = VarianceThreshold(threshold=0.01)
X_fe_selected = pd.DataFrame(
    selector.fit_transform(X_fe_selected),
    columns=X_fe_selected.columns[selector.get_support()],
    index=X_fe_selected.index
)
print(f"   Düşük varyans sonrası: {X_fe_selected.shape[1]} feature")

# 2. Yüksek korelasyonlu feature'ları çıkar
print("\n2. Yüksek korelasyonlu feature'lar çıkarılıyor...")
corr_matrix = X_fe_selected.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
if to_drop:
    X_fe_selected = X_fe_selected.drop(columns=to_drop)
    print(f"   Çıkarılan feature'lar: {to_drop}")
    print(f"   Yüksek korelasyon sonrası: {X_fe_selected.shape[1]} feature")
else:
    print("   Yüksek korelasyonlu feature yok! ")

# 3. Feature Importance ile seçim
print("\n3. Feature Importance'a göre seçim yapılıyor...")
# Önce model eğitip importance'ları alalım
model_temp = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1
)
model_temp.fit(X_fe_selected, y)

importance_df = pd.DataFrame({
    'feature': X_fe_selected.columns,
    'importance': model_temp.feature_importances_
}).sort_values('importance', ascending=False)

print(f"   Top 15 feature:")
print(importance_df.head(15)[['feature', 'importance']].to_string(index=False))

# Top 20 feature'ı seç
top_n = 20
top_features = importance_df.head(top_n)['feature'].tolist()
X_fe_final = X_fe_selected[top_features]

print(f"\n   En önemli {top_n} feature seçildi.")
print(f"   Final feature sayısı: {X_fe_final.shape[1]}")

print(f"\n Feature Selection tamamlandı!")
print(f"   Başlangıç: {X_fe2.shape[1]} feature")
print(f"   Final: {X_fe_final.shape[1]} feature")
print(f"   Çıkarılan: {X_fe2.shape[1] - X_fe_final.shape[1]} feature")

In [ ]:
# Final performans ölçümü
print("=" * 50)
print("FINAL PERFORMANS ÖLÇÜMÜ (Feature Selection Sonrası)")
print("=" * 50)
final_auc = evaluate_features(X_fe_final, y, 'Final (Seçilmiş Feature\'lar)', faz2_auc)

print(f"\n{'='*50}")
print("ÖZET")
print(f"{'='*50}")
print(f"Baseline (duration ile):     {BASELINE_CV_AUC:.4f}")
print(f"Faz 1 (duration çıkarıldı): {faz1_auc:.4f}")
print(f"Faz 2 (tüm feature'lar):     {faz2_auc:.4f}")
print(f"Final (seçilmiş feature'lar): {final_auc:.4f}")
print(f"\nBaseline'dan fark: {final_auc - BASELINE_CV_AUC:.4f} ({(final_auc - BASELINE_CV_AUC)/BASELINE_CV_AUC*100:.2f}%)")

---
## 5. Sonuçlar ve Özet

### Performans Karşılaştırması

| Aşama | Feature Sayısı | CV AUC | Açıklama |
|-------|----------------|--------|----------|
| **Baseline (duration ile)** | 16 | 0.9239 | Gerçekçi değil (duration production'da bilemeyiz.) |
| **Faz 1** | 18 | 0.7907 | duration çıkarıldı, temel feature'lar eklendi |
| **Faz 2** | 30 | 0.7897 | Tüm yeni feature'lar eklendi |
| **Final (Seçilmiş)** | 20 | 0.7880 | Feature selection sonrası |

### Önemli Bulgular

1. **duration Feature'inin Etkisi:**
   - Baseline'da en önemli feature'lardan biriydi
   - Production'da kullanılamaz (görüşme yapılmadan önce bilinmez)
   - Çıkarılınca performans düştü: 0.9239 → 0.7907 (-14.4%)
   - bu beklediğimiz bir şeydi.

2. **Feature Engineering Etkisi:**
   - Faz 1'de yas grupları, bakiye kategorileri eklendi
   - Faz 2'de mevsimsellik, interaksiyon, ratio feature'ları eklendi
   - Toplam 14 yeni feature türetildi

3. **Feature Selection:**
   - 30 feature'dan 20 feature'a düşürüldü
   - Düşük varyanslı ve yüksek korelasyonlu feature'lar çıkarıldı
   - En önemli 20 feature seçildi

### En Önemli Feature'lar (SON)
1. **month** (356) - Mevsimsellik
2. **day** (355) - Ayın günü
3. **age** (291) - Müşteri yaşı
4. **age_campaign_interaction** (287) - Yaş × Kampanya
5. **balance_per_age** (252) - Yaş başına bakiye

### Sonraki Adımlar

1. **Model Optimizasyonu** (04_model_optimization.ipynb)
   - Hiperparametre optimizasyonu (Optuna)
   - Model karşılaştırması
   - Final model seçimi

2. **Model Değerlendirme** (05_model_evaluation.ipynb)
   - SHAP analizi
   - Business uyumluluk kontrolü
   - Threshold optimizasyonu

### Not

**Gerçekçi bir production modeli için:**
- `duration` feature'i kullanılamaz
- Final AUC: 0.7880 (duration olmadan)
- Bu skor, gerçek dünya senaryosunda kullanılabilir bir modeldir
- Baseline'dan fark: -14.71% (duration'ın etkisi büyük ama gerçekçi değil)